# Foursquare Places API in Python

This notebook demonstrates the use of Python to access Foursquare's Places API to get locations and details of venues. The service is based on the idea of crowdsourcing recommendations and 'checkins' from users of the Foursquare and Swarm mobile apps. To see the service's browser-based front end you can visit the [Foursquare City Guide](https://foursquare.com/city-guide).

In order to use the Foursquare Places API you will need to [create a Foursquare developer account](https://developer.foursquare.com/). You will then need to create a new app. See the [Places API documentation](https://developer.foursquare.com/docs/api) for further details. The default 'Sandbox' account enables you to make 950 regular API calls and 50 premium calls per day. Check details of [Rate Limits](https://developer.foursquare.com/docs/api/troubleshooting/rate-limits) and the [Endpoints Overview](https://developer.foursquare.com/docs/api/endpoints) for further information.

NOTE: You can apply for a free 'Personal' account which enables 99,500 regular calls and 500 premium calls per day. However, be aware that upgrading to ta personal account requires verification with a credit card.

#### Import Dependencies

In [1]:
# Import the Requests library to make HTTP requests and handle responses
import requests
# Import the JSON library to process the API response
import json
# Import JSON normalize from Pandas to flatten JSON
from pandas.io.json import json_normalize
# Import date from datetime for setting and formatting date
from datetime import date
# Import the folium library to map our data
import folium

#### Authentication Credentials

In [2]:
# Enter the details from the Foursquare App you created here
client_id = ''
client_secret = ''

# The API version is a date in YYYYMMDD format. The current or a past date is required.
api_ver = date.today().strftime('%Y%m%d')

#### REST API Details

In [3]:
# Basue URL for Places API endpoints
base_url = 'https://api.foursquare.com/v2/'
#End point group
group = 'venues/'

## Searching for Venues by Location

As Foursquare is a location-based service the primary means of obtaining details of venues is by specifying a location. This can be done in several ways but the appropriate method varies depending on the particular endpoint. There are three alternative methods:
1. Specify Lat/Lon coordinates in WGS84 for the endpoint's `ll` parameter.
2. Provide a place name (e.g. `'Nelson's Column'`) for the endpoint's `near` parameter.
3. Provide a geographic bounding box by specifying two sets of Lat/Lon coordinates in WGS84. The first describes the southwest corner and is placed in the `sw` parameter. The second describes the northeast corner and is entered for the `nw` parameter.

To obtain WGS84 Lat/Lon coordinates you can use [OpenStreetMap](https://www.openstreetmap.org/). Methods 1 and 2 can be used in combination with the `radius` parameter which can be used to focus or expand the search area. The search radius defaults to a city-wide area and has a maximum radius of approximately 100,000 metres (or 2,000 in the case of the Trending Venues endpoint). Method 3 are not valid `ll` or `radius` and have a maximum area of up to 10,000 square kilometers.

The `search`, `explore` and `trending` endpoints in the venue group all support methods 1 and 2. Method 3 is only supported by the `search` endpoint. See [Endpoints Overview](https://developer.foursquare.com/docs/api/endpoints).

**NOTE: Specifying a larger area does NOT necessarily result in more results.**

There is a `limit` parameter associated with each of these endpoints which determines the maximum number of results per API call. This parameter has a maximum value of 50 which means that expanding the search area in urban regions simply results in a spatially wider distribution of results. In urban areas making a larger number of call on smaller neighbouring area may help obtain a greater number of results. Be wary of rate limits however. See [Foursquare Venue API & Number of Results](https://stackoverflow.com/questions/8277192/foursquare-venue-api-number-of-results) and [Foursquare API: Getting an exhaustive list of venues in a given area](https://stackoverflow.com/questions/10900896/foursquare-api-getting-an-exhaustive-list-of-venues-in-a-given-area) for more information.

For now we'll focus on the endpoints for Search and Trending Venues.

#### API Endpoints

In [4]:
#Regular API endpoints for the venues group
search_ep = 'search/'
trend_ep = 'trending/'

## Trending Venues

This returns a list of venues near the specified location with the most people currently checked in. See [Get Trending Venues](https://developer.foursquare.com/docs/api/venues/trending) for endpoint parameters and response.

#### API call URL and parameters

In [5]:
# Trending endpoint URL
url = base_url + group + trend_ep

# Trending endpoint parameters
params = dict(
    client_id = client_id,
    client_secret = client_secret,
    v = api_ver,
    #ll = '51.50775,-0.12794',
    near = 'Trafalgar Square',
    radius = 2000,
    limit=50
)

#### API request

In [6]:
# Make the API call 
payload = requests.get(url=url, params=params)
# Store the returned payload
data = json.loads(payload.text)
# View the JSON 'response' object to confirm its structure
#data['response']

#### Process the response

In [7]:
try:
    # Select the venues from the JSON response
    venues_data = data['response']['venues']
    # Flatten the JSON and load into a PANDAS dataframe
    venues_df = json_normalize(venues_data)
    # Filter the dataframe to select required fields
    filtered_columns = ['id',
                        'name',
                        'location.address',
                        'location.city',
                        'location.state',
                        'location.postalCode',
                        'location.country',
                        'location.distance',
                        'location.lat',
                        'location.lng'
                       ]
    # Save the filtered group of columns as a new dataframe and reindex
    venues = venues_df.reindex(columns = filtered_columns)
    # Clean column names for display
    venues.columns = venues.columns.str.replace('location.','')
except:
    print('There are no venues in the API reponse!')

try:
    # Display the new dataframe
    venues
except:
    print('Unable to generate dataframe')

#### Display the new dataframe

In [8]:
# Display the new dataframe
venues

,id,name,address,city,state,postalCode,country,distance,lat,lng
0,4ae5b238f964a52087a121e3,Selfridges & Co,400 Oxford St,London,Greater London,W1A 1AB,United Kingdom,NaN,51.514640,-0.152864
1,4ac518d2f964a5203da720e3,British Museum,Great Russell St,London,Greater London,WC1B 3DG,United Kingdom,NaN,51.519009,-0.126437
2,5cb044142be425002c5682b4,JOE & THE JUICE,20 North Audley St,London,Greater London,W1K 6LX,United Kingdom,NaN,51.513456,-0.153472
3,4ba6419bf964a520b23f39e3,Covent Garden Market,The Piazza,London,Greater London,WC2E 8RF,United Kingdom,NaN,51.511977,-0.122799
4,4ac518d3f964a52076a720e3,Tate Modern,53 Bankside,London,Greater London,SE1 9TG,United Kingdom,NaN,51.507704,-0.099456


#### Visualise the venue locations on a map with Folium

In [9]:
# Set the map's initial centrepoint, zoom level and basemap
folium_map = folium.Map(
    location = [51.50775,-0.12794],
    zoom_start = 13,
    #tiles='Stamen Terrain'
)

search_location = folium.Marker(
    location = [51.50775,-0.12794],
    tooltip = 'Start Point!',
    icon = folium.Icon(color ='green', icon ='info-sign')
).add_to(folium_map)

# Add markers by applying a lambda function along the columns (axis = 1) to add markers 
try:
    venues.apply(
        lambda row:folium.Marker(
            location = [row['lat'], row['lng']],
            icon = folium.Icon(color ='blue', icon ='flag'),
            tooltip = row['name'] + ', ' + row['address'] + ', ' + row['postalCode']
        ).add_to(folium_map), axis = 1)
except:
    print('There are no venues to display!') 

# Draw the map
folium_map

## Search for Venues

This endpoint eturns a list of venues near the specified location. It also provides the opportunity to select matches using an optional search or `query` term. See [Search for Venues](https://developer.foursquare.com/docs/api/venues/search) for endpoint parameters and response.

#### API call URL and parameters

In [10]:
# Search endpoint URL
url = base_url + group + search_ep

#Search endpoint parameters
params = dict(
    client_id = client_id,
    client_secret = client_secret,
    v = '20191010',
    ll = '51.50775,-0.12794',
    #ne = '51.530577,-0.075631',
    #sw = '51.505582,-0.156312',
    #intent = 'browse',
    query = 'McDonalds',
    #near = 'Trafalgar Square',
    radius = 100000,
    limit=50
)

#### API Request

In [11]:
# Make the API call 
payload = requests.get(url=url, params=params)
# Store the returned payload
data = json.loads(payload.text)
# View the JSON 'response' object to confirm its structure
#data['response']

#### Process the response

In [12]:
try:
    # Select the venues from the JSON response
    venues_data = data['response']['venues']
    # Flatten the JSON and load into a PANDAS dataframe
    venues_df = json_normalize(venues_data)
    # Filter the dataframe to select required fields
    filtered_columns = ['id',
                        'name',
                        'location.address',
                        'location.city',
                        'location.state',
                        'location.postalCode',
                        'location.country',
                        'location.distance',
                        'location.lat',
                        'location.lng'
                       ]
    # Save the filtered group of columns as a new dataframe and reindex
    venues = venues_df.reindex(columns = filtered_columns)
    # Clean column names for display
    venues.columns = venues.columns.str.replace('location.','')
except:
    print('There are no venues in the API reponse!')

#### Display the new data frame

In [13]:
# Display the new dataframe
venues

,id,name,address,city,state,postalCode,country,distance,lat,lng
0,57fdeb3e498e21a42a59fcf9,McDonald's,48 Leicester Sq,London,Greater London,WC2H 7LU,United Kingdom,338,51.510291,-0.130617
1,4be16036d816c92817d9efd9,McDonald's,47 Whitehall,City of Westminster,Greater London,SW1A 2BX,United Kingdom,176,51.506311,-0.126868
2,4b44869ff964a52007f625e3,McDonald's,34-35 Strand,London,Greater London,WC2N 5HZ,United Kingdom,260,51.508956,-0.124725
3,4c182e1e6a21c9b6d4bbc897,McDonald's,8-10 Oxford St,London,Greater London,W1D 1AW,United Kingdom,983,51.516415,-0.130700
4,4b7d0e6ff964a5205ead2fe3,McDonald's,25-27 Shaftesbury Ave,London,Greater London,W1V 7HA,United Kingdom,539,51.510915,-0.133838
5,4b2918e5f964a520a79824e3,McDonald's,68-69 St Martins Ln,London,Greater London,WC2N 4JS,United Kingdom,435,51.511629,-0.127151
6,4af889e6f964a520210e22e3,McDonald's,112 High Holborn,Holborn,Greater London,WC1V 6JS,United Kingdom,1264,51.517876,-0.119665
7,4b792866f964a5204ced2ee3,McDonald's,Unit 43 The Colonnade,London,Greater London,SE1 8SW,United Kingdom,1289,51.503121,-0.110887
8,4aeb4302f964a52047c021e3,McDonald's,291B Oxford St,Oxford Circus,Greater London,W1C 2DT,United Kingdom,1380,51.514950,-0.144166
9,4c3c79f35a10b713ae90a34d,McDonald's,185-187 Oxford St,London,Greater London,W1D 1BP,United Kingdom,1158,51.515580,-0.138953


#### Visualise the venue locations on a map with Folium

In [14]:
# Set the map's initial centrepoint, zoom level and basemap
folium_map = folium.Map(
    location = [51.50775,-0.12794],
    zoom_start = 13,
    #tiles='Stamen Terrain'
)

search_location = folium.Marker(
    location = [51.50775,-0.12794],
    tooltip = 'Start Point!',
    icon = folium.Icon(color ='green', icon ='info-sign')
).add_to(folium_map)

# Add markers by applying a lambda function along the columns (axis = 1) to add markers 
try:
    venues.apply(
        lambda row:folium.Marker(
            location = [row['lat'], row['lng']],
            icon = folium.Icon(color ='blue', icon ='flag'),
            tooltip = row['name'] + ', ' + row['address'] + ', ' + row['postalCode']
        ).add_to(folium_map), axis = 1)
except:
    print('There are no venues to display!') 

# Draw the map
folium_map

## Request Venue Details - PREMIUM API Endpoint

This endpoint gives the full details about a venue including location, hours, popular times of data, stats about checkins and more. See [Get Details of a Venue](https://developer.foursquare.com/docs/api/venues/details) for for endpoint parameters and response.

**NOTE: This is a premium API endpoint so the number of call you can make to the endpoint are more limited.**

#### API call URL and parameters

#### In the case of the Venue Details endpoint the main requirement is the `venueID`

In [15]:
# Venue details endpoint URL
url = 'https://api.foursquare.com/v2/venues/'
# VenueID
venueID = '4c9c6a059c48236a1cb14dee'

# Trending endpoint parameters
params = dict(
    client_id = client_id,
    client_secret = client_secret,
    v = api_ver,
)

#### API request

In [16]:
# Search endpoint URL
url = base_url + group + venueID

payload = requests.get(url=url, params=params)
data = json.loads(payload.text)
# View the JSON 'response' object to confirm its structure
#data['response']

#### Process the response

In [17]:
try:
    # Select the venues from the JSON response
    venues_data = data['response']['venue']
    # Flatten the JSON and load into a PANDAS dataframe
    venues_df = json_normalize(venues_data)
    # Filter the dataframe to select required fields
    filtered_columns = ['id',
                        'name',
                        'location.address',
                        'location.city',
                        'location.state',
                        'location.postalCode',
                        'location.country',
                        'location.distance',
                        'location.lat',
                        'location.lng',
                        'stats',
                        'likes',
                        'rating',
                        'ratingColor',
                        'ratingSignals',
                        'hereNow',
                        'hours',
                        'description',
                        'canonicalUrl',
                       ]
    # Save the filtered group of columns as a new dataframe and reindex
    venues = venues_df.reindex(columns = filtered_columns)
    # Clean column names for display
    venues.columns = venues.columns.str.replace('location.','')
except:
    print('There are no venues in the API reponse!')

#### Display the new dataframe

In [18]:
# Display the new dataframe
venues

,id,name,address,city,state,postalCode,country,distance,lat,lng,stats,likes,rating,ratingColor,ratingSignals,hereNow,hours,description,canonicalUrl
0,4c9c6a059c48236a1cb14dee,Piccadilly Circus,Piccadilly,London,Greater London,W1D 1NN,United Kingdom,NaN,51.509961,-0.134652,NaN,NaN,9.0,00B551,8199,NaN,NaN,Piccadilly Circus is famous around the world f...,https://foursquare.com/v/piccadilly-circus/4c9...
